# Face Recognition Software - LiveFeed

## Register A Face

In [ ]:
import tkinter
import tkinter.simpledialog
import cv2
import PIL.Image, PIL.ImageTk
import time


class App:
 def __init__(self, window, window_title, video_source=0):
     self.window = window
     self.window.title(window_title)
     self.video_source = video_source

     # open video source (by default this will try to open the computer webcam)
     self.vid = MyVideoCapture(self.video_source)

     # Create a canvas that can fit the above video source size
     self.canvas = tkinter.Canvas(window, width = self.vid.width, height = self.vid.height)
     self.canvas.pack()

     # Button that lets the user take a snapshot
     self.btn_snapshot=tkinter.Button(window, text="Register Face", width=50, command=self.snapshot)
     self.btn_snapshot.pack(anchor=tkinter.CENTER, expand=True)

     # After it is called once, the update method will be automatically called every delay milliseconds
     self.delay = 10
     self.update()
     self.window.mainloop()

 def snapshot(self):
     face_encodings = {}
     ret, frame = self.vid.get_frame()
     y2,x2,y1,x1 = fr.face_locations(frame)[0]
     cropped = frame[y2-30:y1, x1 : x2]
     cropped = cv2.cvtColor(cropped, cv2.COLOR_RGB2BGR)
     if ret:
         name = tkinter.simpledialog.askstring("Face Details", "Please Enter the Name")
         cv2.imwrite(f"Faces/images/{name}" + ".jpg", cropped)
         
         try:
            with open('Faces/faces_dataset.dat', 'rb') as f:
                face_encodings = pickle.load(f)
         except:
            pass
        
         face_encodings[name] = fr.face_encodings(cropped)[0]   
         with open('Faces/faces_dataset.dat', 'wb') as f:
            pickle.dump(face_encodings, f)
    
 def update(self):
     # Get a frame from the video source
     ret, frame = self.vid.get_frame()
     detect_frame = detect_face(frame)
     
     if detect_frame is not None:
        frame = detect_frame
     
     if ret:
         self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(frame))
         self.canvas.create_image(0, 0, image = self.photo, anchor = tkinter.NW)

     self.window.after(self.delay, self.update)


class MyVideoCapture:
 def __init__(self, video_source=0):
     # Open the video source
     self.vid = cv2.VideoCapture(video_source)
     if not self.vid.isOpened():
         raise ValueError("Unable to open video source", video_source)

     # Get video source width and height
     self.width = self.vid.get(cv2.CAP_PROP_FRAME_WIDTH)
     self.height = self.vid.get(cv2.CAP_PROP_FRAME_HEIGHT)

 def get_frame(self):
     if self.vid.isOpened():
         ret, frame = self.vid.read()
         if ret:
             # Return a boolean success flag and the current frame converted to BGR
             return (ret, cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
         else:
             return (ret, None)
     else:
         return (ret, None)

 # Release the video source when the object is destroyed
 def __del__(self):
     if self.vid.isOpened():
         self.vid.release()

# Create a window and pass it to the Application object
App(tkinter.Tk(), "Tkinter and OpenCV")

In [266]:
def recognize_all(img):
    cpy = img.copy()
    try:
        with open('Faces/faces_dataset.dat', 'rb') as f:
            face_encodings = pickle.load(f)
        face_names = list(face_encodings.keys())
        face_encodings = np.array(list(face_encodings.values()))

        new_faces_coordinates = fr.face_locations(img)

        for i in new_faces_coordinates:
            name = 'Unknown'
            y2,x2,y1,x1 = i
            cropped = img[y2:y1, x1:x2]
            unknown_enc = fr.face_encodings(cropped)
            matches = fr.compare_faces(face_encodings, unknown_enc)

            if True in matches:
                match_index = matches.index(True)
                name = face_names[match_index]
                
            cpy = cv2.rectangle(cpy, (x1,y1), (x2, y2), (0,255,255), 1)
            cpy = cv2.putText(cpy,name,(x1,y2), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2,cv2.LINE_AA)
    except:
        pass
    return cpy

In [269]:
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
# cap.set(cv2.CAP_PROP_FPS, 30) 

while True:
#     print(cap.get(cv2.CAP_PROP_FPS))
    ret,frame = cap.read()
    detect_frame = recognize_all(frame)
#     recognise_face()
    if detect_frame is not None:
        cv2.imshow('Face Recognition',detect_frame)
    else:
        cv2.imshow('Face Recognition', frame)

    if cv2.waitKey(1) in (ord('q'),27):
        break
cap.release()
cv2.destroyAllWindows()

# -----------------------------*---------------------------------*-----------------------------

In [1]:
import face_recognition as fr
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tkinter import *

### Preprocessing the Code###

In [2]:
def display(img):
    fig = plt.figure(figsize = (10,8))
    ax = fig.add_subplot(111)
    ax.imshow(img)
    plt.show()

In [3]:
img = fr.load_image_file("../images/known/himanshu.jpg")

In [4]:
img_fake = fr.load_image_file("../images/car.jpg")

In [ ]:
display(img)

In [9]:
def detect_face(img):
    cpy = img.copy()
    faces_detected = fr.face_locations(img)
    print(faces_detected)
    for i in range(len(faces_detected)):
        y2,x2,y1,x1 = faces_detected[i]
        cpy = cv2.rectangle(cpy, (x1,y1), (x2, y2), (0,255,255), 1)
    return cpy

In [13]:
def show_landmarks(img):
    image = img.copy()
    try:
        dict_locations = fr.face_landmarks(image)[0]
        lst_locations = list(dict_locations.values())
        for x in range(len(lst_locations)):
            for y in range(1,len(lst_locations[x])):
                cv2.line(image, lst_locations[x][y-1], lst_locations[x][y], (255,255,255), 1)
        return image
    except:
        pass

In [ ]:
cap = cv2.VideoCapture(0)
while True:
    ret,frame = cap.read()
    detect_frame = show_landmarks(detect_face(frame))
    if detect_frame is not None:
        cv2.imshow('Face Recognition',detect_frame)
    else:
        cv2.imshow('Face Recognition', frame)
    if cv2.waitKey(1) in (ord('q'),27):
        break
cap.release()
cv2.destroyAllWindows()

In [10]:
def recognise_face(train, test):
    biden_encoding = fr.face_encodings(train)[0]
    unknown_encoding = fr.face_encodings(test)[0]
    print(biden_encoding)
    results = fr.compare_faces([biden_encoding], unknown_encoding)
    print(results)

In [11]:
train_img = img.copy()
test_img = fr.load_image_file("../images/unknown/test.jpg")

In [ ]:
recognise_face(train_img,test_img)

### Face Recognition Image Example

In [161]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import face_recognition as fr
import pickle

In [188]:
with open('Faces/faces_dataset.dat', 'rb') as f:
    face_encodings = pickle.load(f)

In [189]:
face_names = list(face_encodings.keys())
face_encodings = np.array(list(face_encodings.values()))

In [190]:
unknown_image = fr.load_image_file("Faces/images/ratan.jpg")
unknown_encoding = fr.face_encodings(unknown_image)
result = fr.compare_faces(face_encodings, unknown_encoding)

In [193]:
names_with_result = list(zip(face_names, result))
print(names_with_result)
name = list(filter(lambda x: x[1] == True, names_with_result))[0][0]

[('himanshu', False), ('ratan', True)]
